### Notebook 1 — Limpieza y preparación de datos Listado Licencias

In [34]:
import numpy as np
import pandas as pd

pd.options.mode.copy_on_write = True # CoW por defecto a partir de Pandas 3.0.0

# configuración de visualización

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [35]:
# carga de data set

df_original = pd.read_csv(
    "./datasets/ListadoLicencias_2024y2025.csv",
    low_memory=False
)
#El dataset se cargó utilizando low_memory=False 
#con el fin de garantizar una inferencia consistente de los tipos de datos, evitando errores derivados de columnas con valores heterogéneos.

In [36]:
df = df_original.copy()

In [37]:
# Normalización nombres de columnas para eliminar espacios
# y evitar errores por inconsistencias en los encabezados.

df.columns = df.columns.str.strip().str.lower()

In [38]:
print("Dataset cargado correctamente")
print("Dimensiones:", df.shape)


Dataset cargado correctamente
Dimensiones: (18915, 39)


In [39]:
# verificacion inicial

print("\n--- TIPOS DE DATOS ---")
print(df.dtypes)

print("\n--- VALORES NULOS (%) ---")
print((df.isnull().mean() * 100).round(2))

print("\n--- DUPLICADOS ---")
print(df.duplicated().sum())


--- TIPOS DE DATOS ---
nif                               object
nombre                            object
apellido1                         object
apellido2                         object
sexo                              object
fechanacimiento                   object
poblaciondomicilio                object
descripcionprovinciadomicilio     object
codigopostaldomicilio             object
telefono1                         object
telefono2                         object
email                             object
estado                             int64
fechaaltasolicitud                object
numeroclub                        object
descripcionentidad                object
temporada                          int64
modalidad                         object
categoria                         object
complementos                      object
importecomplemento                 int64
importefmm                       float64
importeseguro                    float64
importefederacionesp             

In [40]:
#Eliminación de columnas 100% nulas

nulos = df.isna().mean()
cols_100_nulas = nulos[nulos == 1].index

df.drop(columns=cols_100_nulas, inplace=True)



In [41]:
print("\n--- VALORES NULOS (%) ---")
print((df.isnull().mean() * 100).round(2))


--- VALORES NULOS (%) ---
nif                               0.00
nombre                            0.00
apellido1                         0.00
apellido2                         0.10
sexo                              0.00
fechanacimiento                   0.00
poblaciondomicilio                0.00
descripcionprovinciadomicilio     0.00
codigopostaldomicilio             0.00
telefono1                         0.01
telefono2                        73.77
email                             0.03
estado                            0.00
fechaaltasolicitud                0.00
numeroclub                        0.00
descripcionentidad                0.00
temporada                         0.00
modalidad                         0.00
categoria                         0.00
complementos                     73.14
importecomplemento                0.00
importefmm                        0.00
importeseguro                     0.00
importefederacionesp              0.00
importedescuento                  0.0

In [42]:
# conversion de fechas 
# Conversión de variables de fecha a formato datetime
# para facilitar análisis temporal y detección de errores.

columnas_fecha = [
    'fechanacimiento',
    'fechaaltasolicitud',
]

for col in columnas_fecha:
    df[col] = pd.to_datetime(df[col], errors='coerce')


In [43]:
# Numéricos
# Se garantiza la coherencia de las variables numéricas,
# convirtiendo valores inválidos en NaN.

columnas_numericas = df.select_dtypes(include=["int64", "float64"]).columns

for col in columnas_numericas:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [44]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18915 entries, 0 to 18914
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   nif                            18915 non-null  object        
 1   nombre                         18915 non-null  object        
 2   apellido1                      18915 non-null  object        
 3   apellido2                      18896 non-null  object        
 4   sexo                           18915 non-null  object        
 5   fechanacimiento                18915 non-null  datetime64[ns]
 6   poblaciondomicilio             18915 non-null  object        
 7   descripcionprovinciadomicilio  18915 non-null  object        
 8   codigopostaldomicilio          18915 non-null  object        
 9   telefono1                      18914 non-null  object        
 10  telefono2                      4962 non-null   object        
 11  email          

In [45]:
# Limpieza básica de texto para eliminar espacios innecesarios
# sin alterar el significado de las categorías.


columnas_texto = df.select_dtypes(include=["object"]).columns

for col in columnas_texto:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
    )


### TRANSFORMACIONES ANALÍTICAS INICIALES

### Normalización de la variable sexo

Durante la inspección inicial se detectó que la variable **sexo** estaba codificada en origen
mediante las letras **“H”** y **“M”**.  
En el proceso de exploración se identificó una reasignación previa inconsistente, por lo que
se decidió partir nuevamente de los datos originales y aplicar una **normalización semántica**
controlada, con el objetivo de preservar la correcta distribución de la variable y garantizar
la coherencia del análisis posterior.


In [46]:
# Normalización de la variable 'sexo' para unificar categorías

df['sexo'] = df_original['Sexo'].copy()

# Normalización básica
df['sexo'] = (
    df['sexo']
    .astype(str)
    .str.strip()
    .str.upper()
)

# Mapeo SEMÁNTICO CORRECTO
df['sexo'] = df['sexo'].replace({
    'H': 'MASCULINO',
    'M': 'FEMENINO'
})

In [47]:
df['sexo'].value_counts(dropna=False)


sexo
MASCULINO    12677
FEMENINO      6238
Name: count, dtype: int64

In [48]:
# Creación de la variable edad

df['edad'] = (
    (pd.Timestamp.now() - df['fechanacimiento'])
    .dt.days // 365
)


### Creación de la variable edad

Se crea la variable `edad` a partir de la fecha de nacimiento con el objetivo
de disponer de una variable numérica directamente interpretable y analizable.
La edad permite caracterizar el perfil del federado, facilitar comparaciones
entre grupos y apoyar el análisis descriptivo y estadístico, reduciendo la
complejidad que supone trabajar directamente con fechas.


In [49]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18915 entries, 0 to 18914
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   nif                            18915 non-null  object        
 1   nombre                         18915 non-null  object        
 2   apellido1                      18915 non-null  object        
 3   apellido2                      18915 non-null  object        
 4   sexo                           18915 non-null  object        
 5   fechanacimiento                18915 non-null  datetime64[ns]
 6   poblaciondomicilio             18915 non-null  object        
 7   descripcionprovinciadomicilio  18915 non-null  object        
 8   codigopostaldomicilio          18915 non-null  object        
 9   telefono1                      18915 non-null  object        
 10  telefono2                      18915 non-null  object        
 11  email          

 El dataset queda preparado para el análisis exploratorio,sin haber eliminado variables por criterios analíticos.

### Análisis de cardinalidad de variables categóricas

Se analiza la cardinalidad de las variables categóricas una vez finalizada
la limpieza del dataset, con el objetivo de identificar variables con un
número elevado de categorías que puedan requerir agrupación o exclusión
en fases posteriores del análisis exploratorio.


In [50]:
# Análisis de cardinalidad de variables categóricas

cardinalidad = (
    df.select_dtypes(include="object")
      .nunique()
      .sort_values()
)

cardinalidad


sexo                                 2
envío tarjeta                        2
cuaderno19                           2
bloqueado envíos                     2
tipotarjeta                          4
alta                                 8
categoria                           10
modalidad                           11
discapacidad                        39
complementos                        41
descripcionprovinciadomicilio       52
numeroclub                         153
descripcionentidad                 158
niffamiliar                        404
codigopostaldomicilio              926
poblaciondomicilio                1030
nombre                            2671
telefono2                         4503
apellido1                         5128
apellido2                         5208
ncuentabancaria                   7069
direcciondomicilio               17548
telefono1                        17960
email                            17991
acceso formulario                18909
nif                      

### Interpretación de la cardinalidad

El análisis de cardinalidad muestra una clara diferenciación entre variables
analíticas y variables identificativas o administrativas. Las variables con
cardinalidad elevada corresponden principalmente a datos personales o de
localización detallada, por lo que no se utilizarán en el análisis exploratorio
ni en la contrastación de hipótesis, aunque se mantienen en el dataset limpio
por motivos de trazabilidad.

Las variables con cardinalidad baja o media se consideran adecuadas para su uso
en análisis descriptivos y comparativos en fases posteriores.


In [52]:
# Guardado del dataset de licencias limpio
# Este dataset se utilizará en el análisis exploratorio conjunto
# y en la contrastación de hipótesis.


ruta_licencias_limpio = "./datasets/Licencias_2024y2025_limpio.csv"

df.to_csv(
    ruta_licencias_limpio,
    index=False,
    encoding="utf-8-sig"
)



Con la finalización de este notebook se deja consolidado el dataset de
licencias completamente limpio y normalizado. Las decisiones de limpieza
y análisis de cardinalidad han sido documentadas, y el conjunto de datos
se guarda para su uso en análisis exploratorios y comparativos posteriores.
